need to add instructions on 
* install bleeding edge lasagne & theano
* are we to assume that if they can open the notebook they are good?

## A pure python neural network model of reflex conditioning in animal. 

This model uses the following packages:
* [Lasagne](http://lasagne.readthedocs.io/en/stable/index.html) :: A neural network framework built on Theano
* [Numpy](http://www.numpy.org/) :: Scientific computing packaging
* [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) :: Data manipulation, munging, formatting library
* [Theano](http://www.deeplearning.net/software/theano/) :: Deep learning framework
* [Matplotlib.pyplot](https://matplotlib.org/api/pyplot_api.html) :: Visualization

Begin by importing the necessary packages

In [1]:
import lasagne
import theano
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import theano.tensor as T

*Note: If a warning DEPRACATED populates, set a USER variable defined as THEANO_FLAGS to an empty string.* 

Define constansts for the model

In [2]:
# ######################## Define Constants #################################
N_CS = 5
N_CONTEXT = 10
N_SAMPLES = 25
N_BATCHES = 250
N_SIMS = 20
output_file = 'output.xlsx'

The below function builds the dataset or input vector.  The dataset is a 2-dimensional binary array consisting of an initial 5 element CS, or *conditioned stimulus* portion and a 10 element context portion.  The length of each piece, and thus the dataset, is controlled using constants defined above. The `cs[rand_num][0] = 1.0` portion sets the value of the 1st element of a random vector within the array to 1, this correlates to *stimulus present*.

In [4]:
def build_dataset(n_cs=N_CS, n_context=N_CONTEXT, n_samples=N_SAMPLES):
    # build out cs portion of input var
    cs = [[0 for i in range(n_cs)] for j in range(n_samples)]
    rand_num = np.random.randint(0, high=len(cs))
    cs[rand_num][0] = 1.0

    # build out context portion of input var
    context = [float(np.random.randint(0, high=2)) for i in range(n_context)]

    # build input var
    input_var = []
    for array_item in cs:
        input_var.append(array_item + context)
    return np.asarray(input_var)

Next, defining a function to build targets for a loss function within the model.  The previously built dataset is used a reference point when building the targets to mitigate any errors while running the loss function.  The `cs_index = targets.index(1.)` saves the index of the *stimulus present* vector within the larger dataset array for use in later parts of the application. 

In [6]:
def build_targets(input_var):
    targets = []
    for item in input_var:
        if np.any(item[0] == 1.0):
            targets.append(1.0)
        else:
            targets.append(0.0)
    cs_index = targets.index( 1.)
    return np.asarray(targets), cs_index

On to building the network...

I choose to use lasagne library because of its restraint in abstracting away all of the lower level theano functionality.  Within the context of the experiment, it was not only important to see the activations and weights at the layer level, but also to be able to grab and manipulate them.  Other popular neural net libraries do not provide this acccess out of the box.   

Because the cortical network has 2 different training points, i.e. the *lower layer* weights with the hippocampal hidden layer activations, and the *upper layer* weights with the targets data outlined above; each piece of the cortical network had to built separately. This initial function defines the architecture of the lower cortical network.

In [8]:
# ################## Build Lower Cortical Network #############################
def build_cort_low_net(input_var=None):
    l_input = lasagne.layers.InputLayer(shape=(None, 15),
                                        input_var=input_var)
    l_output = lasagne.layers.DenseLayer(
                l_input,
                num_units=40,
                W=lasagne.init.Uniform(range=3.0),
                nonlinearity=lasagne.nonlinearities.rectify)
    return l_output

This network consists of an **Input Layer** accepting a dataset of shape `(None, 15)` where the *None* parameter indicates the size of that dimension is not fixed.  This is followed by a fully connected **Dense Layer** with 40 nodes and a *rectify* activation function.  The weights matrix values are initialized as a random number between *-3 and 3* using [init.Uniform](https://lasagne.readthedocs.io/en/latest/modules/init.html#lasagne.init.Uniform) method.  

The upper cortical network is defined with similar layers to the lower cortical network with the following differences.
1. The **Input Layer** accepts input with shape `(None, 40`) matching the output of the lower cortical network
2. The **Dense Layer** consists of a single node, matching the desired network output
3. A *sigmoid* activation function is used. 

In [9]:
# ################## Build Upper Cortical Network #############################
def build_cort_up_net(input_var=None):
    l_input = lasagne.layers.InputLayer(shape=(None, 40),
                                        input_var=input_var)
    l_output = lasagne.layers.DenseLayer(
                l_input,
                num_units=1,
                W=lasagne.init.Uniform(range=3.0),
                nonlinearity=lasagne.nonlinearities.sigmoid)
    return l_output